In [1]:
import os
import json
import pdal
import subprocess
import glob
from os.path import join, isdir

This code in the next cell will mergd all the laz files in a directory into one (merged_laz)

In [5]:
laz_dir = r"Sites/RCEW/IS2_Data/" #change to the laz folder
out_fp = "merged.laz"
input_files = glob.glob(join(laz_dir, '*.las'))
merge_cmd = ['pdal', 'merge'] + input_files + [join(laz_dir, out_fp)]
subprocess.run(merge_cmd, check=True)

usage: pdal merge [options] files
standard options:
  --developer-debug  Enable developer debug (don't trap exceptions)
  --label            A string to label the process with
  --driver           Override reader driver
  --help             Print help and exit
options:
  --files, -f  input/output files

For more information, see the full documentation for PDAL at http://pdal.io/



PDAL: Must specify an input and output file.



CalledProcessError: Command '['pdal', 'merge', 'Sites/RCEW/IS2_Data/RCEW-ICESat2-ATL03/merged.laz']' returned non-zero exit status 255.

The code in the next cell will perform ground segmentation on the merged laz file and rasterize

In [4]:
import json
import pdal

# Define the paths to the input LAZ file and the output ground LAZ file
input_laz_file = 'merged.laz' 
output_ground_laz_file = 'ground.laz'

# Define the JSON pipeline for filtering and processing
grd_filter_json = '''
{
  "pipeline":[
    {
      "type":"readers.las",
      "filename": "%s"
    },
    {
      "type":"filters.reprojection",
      "out_srs":"EPSG:26911"
    },
    {
      "type":"filters.assign",
      "assignment":"Classification[:]=0"
    },
    {
      "type":"filters.elm"
    },
    {
      "type":"filters.outlier"
    },
    {
      "type":"filters.smrf",
      "ignore":"Classification[7:7]",
      "slope":0.2,
      "window":16,
      "threshold":0.45,
      "scalar":1.2
    },
    {
      "type":"filters.range",
      "limits":"Classification[2:2]"
    },
    {
      "type":"writers.las",
      "filename": "%s"
    }
  ]
}
'''

to_raster_json = '''
{
    "pipeline": [
        "%s",
        {
            "filename": "dem.tif",
            "gdaldriver": "GTiff",
            "output_type": "all",
            "resolution": "1.0",
            "type": "writers.gdal"
        }
    ]
}
'''

# Load the JSON pipeline configurations
grd_filter_pipeline = json.loads(grd_filter_json % (input_laz_file, output_ground_laz_file))
to_raster_pipeline = json.loads(to_raster_json % output_ground_laz_file)

# Run the filtering pipeline
grd_filter_pipeline = pdal.Pipeline(json.dumps(grd_filter_pipeline))
grd_filter_pipeline.execute()

# Run the raster conversion pipeline
to_raster_pipeline[0] = output_ground_laz_file

to_raster_pipeline = pdal.Pipeline(json.dumps(to_raster_pipeline))
to_raster_pipeline.execute()


2550370